In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
def nonlinear_system_with_disturbance(x, u, disturbance):
    """System dynamics with added disturbance."""
    dx1 = x[1]
    dx2 = -x[0] + u + disturbance  # Disturbance adds randomness to the waveform
    return np.array([dx1, dx2])

def adaptive_estimation_disturbance_observer(x_true, x_hat, u, a):
    """Adaptive DOB with estimation law, simplified."""
    x_delta = x_hat - x_true
    exp_term = np.exp(a * dt) - 1
    d_hat = -a * x_delta / exp_term
    dx_hat = np.array([
        x_true[1] + d_hat[0] - a * x_delta[0],
        -x_true[0] + u + d_hat[1] - a * x_delta[1]
    ])

    return dx_hat

In [ ]:
# Simulation parameters
t_max = 50.0
dt = 0.01
time = np.arange(0, t_max, dt)
x_true = np.zeros((len(time), 2))
x_hat = np.zeros((len(time), 2))
u = 1  # Constant control input
a = 100  # DOB parameter, needs tuning based on the system response

# Initial states
x_true[0] = np.array([1.0, 0.0])
x_hat[0] = np.array([0.9, 0.0])
disturbance = np.random.normal(0, 0.01, len(time))  # Random disturbances

# Simulation loop
for i in range(1, len(time)):
    x_true[i] = x_true[i-1] + nonlinear_system_with_disturbance(x_true[i-1], u, disturbance[i-1]) * dt
    x_hat[i] = x_hat[i-1] + adaptive_estimation_disturbance_observer(x_true[i-1], x_hat[i-1], u, a) * dt

# Plotting
plt.figure(figsize=(10, 8))
plt.subplot(2, 1, 1)
plt.plot(time, x_true[:, 0], 'r-', label='True State (x1)')
plt.plot(time, x_hat[:, 0], 'b--', label='Estimated State (x1)')
plt.xlabel('Time (s)')
plt.ylabel('State x1')
plt.legend()
plt.title('Comparison of True and Estimated States for x1')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(time, x_true[:, 1], 'r-', label='True State (x2)')
plt.plot(time, x_hat[:, 1], 'b--', label='Estimated State (x2)')
plt.xlabel('Time (s)')
plt.ylabel('State x2')
plt.legend()
plt.title('Comparison of True and Estimated States for x2')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# with different sampling rates

# Simulation parameters
t_max = 50.0
dt_data = 0.01
dt_distu = 0.02
time_data = np.arange(0, t_max, dt_data)
time_distu = np.arange(0, t_max, dt_distu)
x_true = np.zeros((len(time_data), 2))
x_hat = np.zeros((len(time_distu), 2))
u = 1  # Control input
a = 0.02 # DOB parameter, needs tuning

# Initial conditions
x_true[0] = np.array([1.0, 0.0])
x_hat[0] = np.array([0.9, 0.0])
disturbance = np.random.normal(0, 0.01, len(time_data))  # Random disturbance

# Generate true state
for i in range(1, len(time_data)):
    x_true[i] = x_true[i-1] + nonlinear_system_with_disturbance(x_true[i-1], u, disturbance[i-1]) * dt_data

# Interpolate x_true for disturbance observer times
interp_func = interp1d(time_data, x_true, axis=0)
x_true_interpolated = interp_func(time_distu)

# Estimate states with disturbance observer
for i in range(1, len(time_distu)):
    x_hat[i] = x_hat[i-1] + adaptive_estimation_disturbance_observer(x_true_interpolated[i-1], x_hat[i-1], u, a) * dt_distu 

# Plotting
plt.figure(figsize=(12, 8))
# Plot for state x1
plt.subplot(2, 1, 1)
plt.plot(time_data, x_true[:, 0], 'b-', label='True State x1')
plt.plot(time_distu, x_hat[:, 0], 'r--', label='Estimated State x1')
plt.title('State x1 Comparison')
plt.xlabel('Time (s)')
plt.ylabel('State x1')
plt.legend()

# Plot for state x2
plt.subplot(2, 1, 2)
plt.plot(time_data, x_true[:, 1], 'b-', label='True State x2')
plt.plot(time_distu, x_hat[:, 1], 'r--', label='Estimated State x2')
plt.title('State x2 Comparison')
plt.xlabel('Time (s)')
plt.ylabel('State x2')
plt.legend()

plt.tight_layout()
plt.show()